In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.is_available()

## Download CRE Stereo code

In [ ]:
!git clone https://github.com/megvii-research/CREStereo.git

In [ ]:
!pip install -r CREStereo/requirements.txt

## Generate left disparity map

May have to resolve typo in `/content/CREStereo/nets/corr.py:39`:
```
TypeError: pad() got an unexpected keyword argument 'pad_witdth'
```

Also remove color-mapping in `/content/CREStereo/test.py:101`

In [ ]:
# !python3 CREStereo/test.py --model CREStereo/crestereo_eth3d.mge --left CREStereo/img/test/roses_L.png --right CREStereo/img/test/roses_R.png --size 500x338 --output left_disparity.png
!mv left_disparity.png CREStereo/img/test/roses_left_disparity.png


## Generate right disparity map

In [ ]:
# !pip install opencv-python

import cv2
from google.colab.patches import cv2_imshow

'''
Rotate left and right disparity maps
'''
left_img = cv2.imread("CREStereo/img/test/roses_L.png")
rotated_left_img = left_img[:, ::-1]
# cv2_imshow(rotated_left_img)
cv2.imwrite("roses_L_rotated.png", rotated_left_img)

right_img = cv2.imread("CREStereo/img/test/roses_R.png")
rotated_right_img = right_img[:, ::-1]
# cv2_imshow(rotated_right_img)
cv2.imwrite("roses_R_rotated.png", rotated_right_img)

!mv roses_R_rotated.png CREStereo/img/test/roses_R_rotated.png
!mv roses_L_rotated.png CREStereo/img/test/roses_L_rotated.png

'''
Generate rotated version of right disparity map
'''
!python3 CREStereo/test.py --model CREStereo/crestereo_eth3d.mge --left CREStereo/img/test/roses_R_rotated.png --right CREStereo/img/test/roses_L_rotated.png --size 500x338 --output right_disparity_rotated.png


In [ ]:
'''
Correct orientation of right disparity map
'''
right_disp_rotated = cv2.imread("right_disparity_rotated.png")
cv2_imshow(right_disp_rotated)
right_disp = right_disp_rotated[:, ::-1]
cv2_imshow(right_disp)
cv2.imwrite("roses_right_disparity.png", right_disp)

!mv roses_right_disparity.png CREStereo/img/test/roses_right_disparity.png

## Convert disparity maps to grayscale

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

def convertRgbToGrayscale(color_image, side):
    # This solution is based on
    # https://stackoverflow.com/questions/51824718/opencv-jetmap-or-colormap-to-grayscale-reverse-applycolormap
    # create an inverse from the colormap to gray values
    gray_values = np.arange(256, dtype=np.uint8)
    color_values = map(tuple, cv2.applyColorMap(gray_values, cv2.COLORMAP_INFERNO).reshape(256, 3))
    color_to_gray_map = dict(zip(color_values, gray_values))

    # apply the inverse map to the false color image to reconstruct the grayscale image
    gray_image = np.apply_along_axis(lambda bgr: color_to_gray_map[tuple(bgr)], 2, color_image)

    # save reconstructed grayscale image
    cv2.imwrite(f'grayscale_disp_map_{side}.png', gray_image)
    return gray_image

disp = cv2.imread("CREStereo/img/test/roses_left_disparity.png", cv2.IMREAD_UNCHANGED)
left_disp_gray = convertRgbToGrayscale(disp, "left")

cv2_imshow(left_disp_gray)

disp = cv2.imread("CREStereo/img/test/roses_right_disparity.png", cv2.IMREAD_UNCHANGED)
right_disp_gray = convertRgbToGrayscale(disp, "right")

cv2_imshow(right_disp_gray)

# Batch

In [ ]:
!unzip test.zip

In [ ]:
import cv2
from google.colab import files
from google.colab.patches import cv2_imshow

imgs = [
    # "art",
    # "flowerbed",
    "lilies",
    # "statue",
    # "sunflowers",
    # "tree",
    # "trees",
    # "woods",
    # "roses",
    # "rock",
    # "peaches",
    # "mug",
    # "grapes"
]

for img in imgs:
  left_img = cv2.imread(f"test/{img}/{img}_L.png")
  right_img = cv2.imread(f"test/{img}/{img}_R.png")
  x = left_img.shape[0]
  y = left_img.shape[1]
  !python3 CREStereo/test.py --model CREStereo/crestereo_eth3d.mge --left test/{img}/{img}_L.png --right test/{img}/{img}_R.png --size {x}x{y} --output {img}-output/left_disparity.png

  rotated_left_img = left_img[:, ::-1]
  cv2.imwrite(f"test/{img}/{img}_L_rotated.png", rotated_left_img)
  rotated_right_img = right_img[:, ::-1]
  cv2.imwrite(f"test/{img}/{img}_R_rotated.png", rotated_right_img)

  !python3 CREStereo/test.py --model CREStereo/crestereo_eth3d.mge --left test/{img}/{img}_R_rotated.png --right test/{img}/{img}_L_rotated.png --size {x}x{y} --output {img}-output/right_disparity_rotated.png

  right_disp_rotated = cv2.imread(f"{img}-output/right_disparity_rotated.png")
  right_disp = right_disp_rotated[:, ::-1]
  cv2.imwrite(f"{img}-output/right_disparity.png", right_disp)

  !zip -r {img}-output.zip {img}-output
  files.download(f"{img}-output.zip")